In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

from Gaussian import Gaussian
from run_coverage_test import run_coverage_test, plot_coverage, plot_coverage_by_dim
from LinGaussTest import LinGaussInvProb, LinGaussTest
from helper import get_random_corr_mat
from modmcmc import State, BlockMCMCSampler, LogDensityTerm, TargetDensity
from modmcmc.kernels import MarkovKernel, GaussMetropolisKernel, DiscretePCNKernel, mvn_logpdf

In [2]:
rng = np.random.default_rng(532124)
d = 5  # Parameter dimension
n = 10 # Observation dimension
Sig_scale = 0.3
C0_scale = 1.0
Q_scale = 1.0
n_reps = 1000

n_plot_rows = int(max(1, np.ceil(d/4)))

In [3]:
inv_test = LinGaussInvProb(rng, d, n, C0_scale=1.0, Sig_scale=1.0)
Q = Q_scale**2 * get_random_corr_mat(n, rng)
tester = LinGaussTest(inv_test, Q)

In [6]:
tester.estimate_expected_kl(100000)

(np.float64(3.446372138775324), np.float64(5.007925262151802))

In [7]:
tester.calc_expected_kl()

(np.float64(5.1041706507568945), np.float64(7.867292920428153))

In [ ]:
inv_prob, res, probs = run_coverage_test(rng, n_reps, d, n, 
                                         Q_scale=Q_scale, Q_scaler=2,
                                         C0_scale=C0_scale, Sig_scale=Sig_scale)

In [ ]:
inv_prob.plot_marginals(nrows=n_plot_rows)

In [ ]:
fig_joint_cover, ax_joint_cover = plot_coverage(res["ep_cover_joint"], res["eup_cover_joint"], probs=probs)
display(fig_joint_cover)

In [ ]:
fig = plot_coverage_by_dim(res["ep_cover_univariate"], res["eup_cover_univariate"], 
                           probs, q_min=0.1, q_max=0.9, nrows=n_plot_rows)

display(fig)

In [ ]:
n_bins = 30

fig_kl, ax_kl = plt.subplots()
ax_kl.hist(res["ep_kl"], bins=n_bins, label="ep")
ax_kl.hist(res["eup_kl"], bins=n_bins, label="eup", alpha=0.5)
ax_kl.legend()

plt.show()

In [ ]:
n_bins = 30

fig_ekl, ax_ekl = plt.subplots()
ax_ekl.hist(res["ep_expected_kl"], bins=n_bins, label="ep")
ax_ekl.hist(res["eup_expected_kl"], bins=n_bins, label="eup", alpha=0.5)
ax_ekl.legend()

plt.show()

In [ ]:
# Exact inverse problem.
inv_prob = LinGaussInvProb(rng, d, n, C0_scale=1.0, Sig_scale=1.0)
plt_inv_prob = inv_prob.plot_marginals()
display(plt_inv_prob)

In [ ]:
# Surrogate-based inverse problem.
Q = Q_scale**2 * get_random_corr_mat(n, rng)
test = LinGaussTest(inv_prob, Q)

In [ ]:
fig_coverage = test.plot_coverage()
display(fig_coverage)

In [ ]:
n_samp = 50000
fig_hist = test.get_hist_plot(n_samp)
display(fig_hist)

In [ ]:
tester = test.calc_coverage()

In [ ]:
tester["ep"].shape

In [ ]:
for k in tester.keys():
    print(k)

In [ ]:
test.reset_samplers()
fig_rk = test.get_hist_plot(n_samp, include=["ep", "eup", "rk", "rk-pcn"])
display(fig_rk)

In [ ]:
cov_scalers = [0.1, 0.5, 0.9]
figs = []
for s in cov_scalers:
    test.reset_samplers()
    test.samplers["rk-pcn"].kernels[0].proposal.cov = s * test.prior.cov
    fig = test.get_hist_plot(n_samp, include=["ep", "rk-pcn"])
    figs.append(fig)
    display(fig)